# Install and import necessary libraries
# NOTE: Restart of the kernel may be necessary after installing the libraries

In [14]:
!pip install rank_bm25
!pip install tf-keras

ERROR: ld.so: object '/opt/conda/lib/libmkl_def.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_avx2.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_core.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_intel_lp64.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_intel_thread.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_def.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_avx2.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_core.so' from LD_PRE

In [1]:
import torch
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
import json
from pathlib import Path
import re
import random
import os
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder

2025-05-20 08:40:09.033246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747730409.055896    1184 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747730409.062328    1184 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747730409.084553    1184 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747730409.084571    1184 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747730409.084574    1184 computation_placer.cc:177] computation placer alr

# 1) Importing query and collection data

In [2]:
PATH_COLLECTION_DATA = 'subtask_4b/subtask4b_collection_data.pkl'
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [17]:
PATH_QUERY_TRAIN_DATA = 'subtask_4b/subtask4b_query_tweets_train.tsv'
PATH_QUERY_DEV_DATA = 'subtask_4b/subtask4b_query_tweets_dev.tsv'
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [18]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


# 2) Running the BM25 baseline
The following code runs a BM25 baseline.


In [5]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [6]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:100] # @k: how many docs shall the ranked list include?
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [7]:
# Retrieve top50 candidates using the BM25 model

train_pkl_path = 'df_query_train_top100.pkl'
dev_pkl_path = 'df_query_dev_top100.pkl'

if not os.path.exists(train_pkl_path):
    df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
    df_query_train.to_pickle(train_pkl_path)
else:
    df_query_train = pd.read_pickle(train_pkl_path)

if not os.path.exists(dev_pkl_path):
    df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))
    df_query_dev.to_pickle(dev_pkl_path)
else:
    df_query_dev = pd.read_pickle(dev_pkl_path)

# 3) Neural Re-Ranking

In [8]:
# load BM25 pre-ranked query dataframes
train_pkl_path = 'df_query_train_top100.pkl'
dev_pkl_path = 'df_query_dev_top100.pkl'

df_query_dev = pd.read_pickle(dev_pkl_path)
df_query_train = pd.read_pickle(train_pkl_path)

### Rerank using cross encoder:

Fetching the original abstract text based on the BM25 ranking

In [9]:
# Load your .pkl file
df = pd.read_pickle('df_query_dev_top100.pkl')

query_doc_pairs = []
query_ids = []
doc_ids = []

for _, row in df.iterrows():
    query = row['final_query']
    query_id = row['cord_uid']
    topk_docs = row['bm25_topk']

    for doc_id in topk_docs:
        query_doc_pairs.append((query, doc_id))
        query_ids.append(query_id)
        doc_ids.append(doc_id)


df_collection = pd.read_pickle('subtask_4b/subtask4b_collection_data.pkl')
doc_texts = pd.Series(df_collection['abstract'].values, index=df_collection.cord_uid).to_dict()

pairs_with_text = [(query, doc_texts.get(doc_id, "")) for query, doc_id in query_doc_pairs]

In [14]:
def predict_crossencoder(model):
    batch_size = 16
    scores = []
    
    for i in tqdm(range(0, len(pairs_with_text), batch_size)):
        batch = pairs_with_text[i:i+batch_size]
        batch_scores = model.predict(batch)
        scores.extend(batch_scores)
    
    reranked_results = pd.DataFrame({
        'query_id': query_ids,
        'doc_id': doc_ids,
        'score': scores
    }).sort_values(by='score', ascending=False)

    print(reranked_results.head(10))
    return reranked_results

In [15]:
model_name_scibert = "allenai/scibert_scivocab_uncased"
model_name_ms_marco = "cross-encoder/ms-marco-MiniLM-L-6-v2"
model_name_biomed = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

model_scibert = CrossEncoder(model_name_scibert, max_length=512)
model_ms_marco = CrossEncoder(model_name_ms_marco, max_length=512)
model_biomed = CrossEncoder(model_name_biomed, max_length=512)


reranked_results_scibert = predict_crossencoder(model_scibert)
reranked_results_ms_marco = predict_crossencoder(model_ms_marco)
reranked_results_biomed = predict_crossencoder(model_biomed)

reranked_sorted_scibert = reranked_results_scibert.sort_values(['query_id', 'score'], ascending=[True, False])
reranked_sorted_ms_marco = reranked_results_ms_marco.sort_values(['query_id', 'score'], ascending=[True, False])
reranked_sorted_biomed = reranked_results_biomed.sort_values(['query_id', 'score'], ascending=[True, False])

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

...

        query_id    doc_id     score
94019   barindjx  zhg1hvsq  0.713486
18570   h8w4q5ff  tx2krngu  0.710242
44450   o1xsjx2p  zhg1hvsq  0.708625
36449   iwd8nkls  fiicxnty  0.708123
133525  2my9o9fy  zhg1hvsq  0.706270
20928   pgasbdpp  jvo0nk3w  0.706133
70321   vmijl5yu  m1tv7mgm  0.705670
36435   iwd8nkls  td2rr1pf  0.704429
22420   qjqk8xge  1fzqh9mb  0.704170
106042  g5ftdku9  ysu5rwmi  0.703476


  0%|          | 0/8750 [00:00<?, ?it/s]

        query_id    doc_id     score
4000    daj5g788  daj5g788  7.676471
107100  i94boetk  i94boetk  7.255842
101600  m26xr9qr  m26xr9qr  7.184708
93400   v77n1vbf  v77n1vbf  6.459419
34200   0n51j2jo  0n51j2jo  6.365736
118300  0jvoa3ui  0jvoa3ui  6.362935
26600   l16a5nq7  l16a5nq7  6.310903
27300   dnm7s947  dnm7s947  6.124531
40600   66obkrxp  66obkrxp  6.048230
16500   52cub1om  52cub1om  5.991239


  0%|          | 0/8750 [00:00<?, ?it/s]

        query_id    doc_id     score
38371   4ohgr8j7  wgfdd3lm  0.602121
130770  6g4oclln  pa1dt9zt  0.601558
128875  8fnis63j  u5iqcw74  0.600750
24199   k9erix1s  80kxz7yr  0.599603
12285   1f6kq8co  whh34bjb  0.598552
130772  6g4oclln  8pz6131o  0.597854
74287   6gb1za5h  m8xfza1d  0.597732
100442  u5rbvc3i  z94skip6  0.597062
46120   tmdsyzyt  umyojne2  0.596951
12226   1f6kq8co  jn68rbvb  0.596661


In [19]:
topk_per_query_scibert = reranked_sorted_scibert.groupby('query_id')['doc_id'].apply(list).reset_index()
topk_per_query_scibert.rename(columns={'doc_id': 'crossencoder_topk_scibert'}, inplace=True)
df_query_dev = df_query_dev.merge(topk_per_query_scibert, left_on='cord_uid', right_on='query_id', how='left')
#df_query_dev.rename(columns={'crossencoder_topk_y': 'crossencoder_topk_scibert'}, inplace=True)

topk_per_query_ms_marco = reranked_sorted_ms_marco.groupby('query_id')['doc_id'].apply(list).reset_index()
topk_per_query_ms_marco.rename(columns={'doc_id': 'crossencoder_topk_ms_marco'}, inplace=True)
df_query_dev = df_query_dev.merge(topk_per_query_ms_marco, left_on='cord_uid', right_on='query_id', how='left')


topk_per_query_biomed = reranked_sorted_biomed.groupby('query_id')['doc_id'].apply(list).reset_index()
topk_per_query_biomed.rename(columns={'doc_id': 'crossencoder_topk_biomed'}, inplace=True)
df_query_dev = df_query_dev.merge(topk_per_query_biomed, left_on='cord_uid', right_on='query_id', how='left')


df_query_dev.head()

,post_id,tweet_text,cord_uid,query_id_x,crossencoder_topk_scibert,query_id_y,crossencoder_topk_ms_marco,query_id,crossencoder_topk_biomed
0,16,covid recovery: this study from the usa reveal...,3qvh482o,3qvh482o,"[yoiq6cgt, mena480g, 34dmal99, g6elhxzt, xwax7...",3qvh482o,"[tdwkxwod, jwei2g97, atji1xge, 66g5lpm6, es8l2...",3qvh482o,"[8pz6131o, t2gxkxxd, ky5env7t, lcqxnrge, xndph..."
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,r58aohnu,"[zvngy7zz, jg7ycmf9, g5sw7t8x, ey34e59f, gatxu...",r58aohnu,"[r58aohnu, r58aohnu, 8hchfgx6, 0clp6zt6, ropgq...",r58aohnu,"[bs8tdve2, 3jqsmc7p, fd4edh93, ai60maqz, 9hpkm..."
2,73,I recall early on reading that researchers who...,sts48u9i,sts48u9i,"[50dmclk4, 1gbq9j88, mkwgkkoi, ohyhjt2p, stew8...",sts48u9i,"[0j4ind0z, dnm7s947, jfeamjqu, y4m987yn, ex6kt...",sts48u9i,"[whh34bjb, 50dmclk4, 66fi7e69, 2qksxqmb, zu8dg..."
3,93,You know you're credible when NIH website has ...,3sr2exq9,3sr2exq9,"[7clr6hbf, ucpdyk7z, j2oo4hoo, rvzvibbt, mdx41...",3sr2exq9,"[8j3bb6zx, k0f4cwig, sv48gjkk, h8x5cpz4, sv48g...",3sr2exq9,"[j2oo4hoo, dwgxrbag, 7clr6hbf, wqu8dm15, rvzvi..."
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,ybwwmyqy,"[mp1z0269, ijsn8d7b, 4hmecfi0, z35xw2p0, bf9om...",ybwwmyqy,"[ybwwmyqy, rs3umc1x, ouvq2wpq, ybwwmyqy, lzddn...",ybwwmyqy,"[lmo2t4ho, nl1ud0rz, m8xfza1d, i71r4eks, f5g3m..."


# 4) Evaluation
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [13]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [ ]:
# ---- BM25 Baseline ----
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')

print("---- BM25 Baseline ----")
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

---- BM25 Baseline ----
Results on the train set: {1: 0.5731735781529604, 5: 0.625250914183459, 10: 0.6308237901348459}
Results on the dev set: {1: 0.5657142857142857, 5: 0.616095238095238, 10: 0.6224325396825396}


In [20]:
#Cross encoder approach scores

mrr_scores_scibert = get_performance_mrr(df_query_dev, col_gold='cord_uid', col_pred='crossencoder_topk_scibert')
mrr_scores_ms_marco = get_performance_mrr(df_query_dev, col_gold='cord_uid', col_pred='crossencoder_topk_ms_marco')
mrr_scores_biomed = get_performance_mrr(df_query_dev, col_gold='cord_uid', col_pred='crossencoder_topk_biomed')


print("---- Re-Ranking Cross encoder ----")
print(f"MRR scibert on dev set: {mrr_scores_scibert}")
print(f"MRR ms_marco on dev set: {mrr_scores_ms_marco}")
print(f"MRR biomed on dev set: {mrr_scores_biomed}")

---- Re-Ranking Cross encoder ----
MRR scibert on dev set: {1: 0.0, 5: 0.001095238095238095, 10: 0.001943310657596372}
MRR ms_marco on dev set: {1: 0.41928571428571426, 5: 0.4946071428571429, 10: 0.5080779478458051}
MRR biomed on dev set: {1: 0.013571428571428571, 5: 0.029476190476190475, 10: 0.03796513605442177}


## Results documentation

### Implementations of a cross-encoder approach
<br>
<br>

|Model                        | MRR@5 (dev)     |
|-----------------------------|-----------------|
|BM25 (baseline)              |55.20%           |
|SciBERT (Cross-encoding)     |0.0%             |
|MiniLM (Cross-encoding)      |49.46%           |
|BioMed (Cross-encoding)      |2.94%            |

In [ ]:
model_name = "bm25"

df_query_dev['preds'] = df_query_dev[f'{model_name}_topk'].apply(lambda x: x[:5])

In [ ]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')

### Grid search for best parameters

Trying out different combinations of margin and number of negative document samples for training. <br>
Result: **margin of 0.5** seems to work best.

In [ ]:
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [ ]:
from itertools import product
import random, torch, numpy as np

param_grid = {
    "margin":        [0.3, 0.4, 0.5],
    "num_negatives": [1, 2, 4]
}
keys, values = zip(*param_grid.items())

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

for i, (margin, num_neg) in enumerate(product(*values), 1):
    set_seed()
    model_name = f"colB_sciB_m{str(margin)[-1]}_neg{num_neg}"
    print(f"\n🔍 Combination {i}/9 → {model_name}")

    scibert_finetune(
        save_name      = model_name,
        MARGIN         = margin,
        BATCH_SIZE     = 8,
        EPOCHS         = 6,
        LR             = 2e-5,
        num_negatives  = num_neg
    )

    # ---------- Evaluation ----------
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model     = AutoModel.from_pretrained(model_name)
    metadata  = pre_compute_embeddings(model_name)

    df_query_dev = rerank(df_query_dev, metadata, tokenizer, model, model_name)
    mrr_scores   = get_performance_mrr(df_query_dev, "cord_uid", f"{model_name}_topk")
    print("MRR@5:", mrr_scores[5])



🔍 Combination 1/9 → colB_sciB_m3_neg1


Epoch 1:   0%|          | 0/1607 [00:00<?, ?it/s]

Epoch 1 Loss: 121.9535


Epoch 2:   0%|          | 0/1607 [00:00<?, ?it/s]

Epoch 2 Loss: 40.3961


...

Epoch 6:   0%|          | 0/6427 [00:00<?, ?it/s]

Epoch 6 Loss: 31.5070


  0%|          | 0/7718 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

MRR@5: 0.6259285714285714


### 3.1) Baseline: ColBERT architecture with SciBERT
Use a pretrained SciBERT model to:
- embed each query-token
- embed each doc-token (can be pre-computed)

For each query-doc pair:
- calculate match-matrix: each query-token – doc-token pair gets cosine similarity value
- aggregate the score:
    - for each query-token take max cosine similarity value with corresponding doc-tokens
    - sum over all of the max elements

In [ ]:
def get_token_embeddings(text, tokenizer, model, device='cpu'):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)
    attention_mask = inputs['attention_mask'].squeeze(0).bool()
    token_embeddings = token_embeddings[attention_mask]
    return token_embeddings

def build_and_save_doc_embeddings(
    docs_df,
    model_name,
    save_dir,
    max_len=512,
    device="cuda"
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    save_path = Path("doc_embeddings_" + save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    metadata_path = save_path / "metadata.json"
    if metadata_path.exists():
        with open(metadata_path, "r") as f:
            metadata = json.load(f)
    else:
        metadata = {}

    for i, row in tqdm(docs_df.iterrows(), total=len(docs_df)):
        doc_id = row.get("cord_uid", f"doc_{i}")
        file_path = save_path / f"{doc_id}.pt"

        if file_path.exists() and doc_id in metadata:
            continue

        text = str(row.get('title', '')) + " " + str(row.get('abstract', '')) + " Authors: " + str(row.get('authors', ''))

        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=max_len)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        output = model(**inputs)
        token_embeddings = output.last_hidden_state.squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0).bool()
        token_embeddings = token_embeddings[attention_mask]

        n_tokens = token_embeddings.size(0)
        pad_len = max_len - n_tokens

        if pad_len > 0:
            padding = torch.zeros(pad_len, token_embeddings.size(1), device=device)
            token_embeddings = torch.cat([token_embeddings, padding], dim=0)
        else:
            token_embeddings = token_embeddings[:max_len]

        try:
            torch.save(token_embeddings.cuda(), file_path)
        except Exception as e:
            print(f"Fehler beim Speichern von {doc_id}: {e}")
            continue

        metadata[doc_id] = {
            "title": row.get("title", ""),
            "abstract": row.get("abstract", ""),
            "authors": row.get("authors", ""),
            "length": min(n_tokens, max_len),
            "path": str(file_path)
        }

    with open(metadata_path, "w") as f:
        json.dump(metadata, f)

    return metadata


In [ ]:
def pre_compute_embeddings(save_name):
    if not os.path.exists("doc_embeddings_" + save_name + "/metadata.json"):
        metadata = build_and_save_doc_embeddings(df_collection, model_name=model_name, save_dir=save_name, device="cuda")
    else:
        with open("doc_embeddings_" + save_name + "/metadata.json", "r") as f:
            metadata = json.load(f)
    return metadata

In [ ]:
def rerank(df, metadata, tokenizer, model, save_name):
    device = next(model.parameters()).device
    df[save_name + '_scores'] = [[] for _ in range(len(df))]

    doc_embeddings = {}
    for doc_id, data in metadata.items():
        emb = torch.load(data["path"], map_location="cpu")
        doc_embeddings[doc_id] = emb

    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            tweet_text = row['tweet_text']
            pre_ranked_docs = row['bm25_topk']

            q_emb = get_token_embeddings(tweet_text, tokenizer, model).to(device)
            q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)

            scores = []
            for doc in pre_ranked_docs:
                emb = doc_embeddings[doc].to(device)
                length = metadata[doc]["length"]
                d_emb = emb[:length]
                d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)

                sim_matrix = torch.matmul(q_norm, d_norm.T)
                max_sim_per_q = sim_matrix.max(dim=1).values
                score = max_sim_per_q.sum().item()
                scores.append(score)

            df.at[idx, save_name + '_scores'] = scores

    def sort_docs_by_score(row):
        doc_ids = row['bm25_topk']
        scores = row[save_name + '_scores']
        sorted_docs = [doc for doc, _ in sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)]
        return sorted_docs

    df[save_name + '_topk'] = df.apply(sort_docs_by_score, axis=1)
    return df

In [ ]:
# settings for model run:
model_name = "allenai/scibert_scivocab_uncased"
save_name = "scibert_baseline"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# pre-compute embeddings
metadata = pre_compute_embeddings(save_name)

In [ ]:
# re-rank BM25 list for dev data
df_query_dev = rerank(df_query_dev, metadata, tokenizer, model, save_name)

In [ ]:
# re-rank BM25 list for train data
df_query_train = rerank(df_query_train, metadata, tokenizer, model)

### ColBERT w/ fine-tuned SciBERT for docs and CTBERT for queries

In [ ]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from tqdm import tqdm
import json
import random
import torch.nn as nn

# hyperparameter
BATCH_SIZE = 8
EPOCHS = 4
LR = 2e-5
MARGIN = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# load model
query_model_name = "digitalepidemiologylab/covid-twitter-bert"
doc_model_name = "allenai/scibert_scivocab_uncased"

query_tokenizer = AutoTokenizer.from_pretrained(query_model_name)
doc_tokenizer = AutoTokenizer.from_pretrained(doc_model_name)

query_model = AutoModel.from_pretrained(query_model_name).to(DEVICE)
doc_model = AutoModel.from_pretrained(doc_model_name).to(DEVICE)

# projection to same dimensions
query_projection = nn.Linear(1024, 768).to(DEVICE)

# dataset creation
class ColBERTTripletDataset(Dataset):
    def __init__(self, df, metadata, num_negatives=1):
        self.data = []
        self.metadata = metadata
        for _, row in df.iterrows():
            query = row["tweet_text"]
            pos = row["cord_uid"]
            negatives = [doc for doc in row["bm25_topk"] if doc != pos]
            if negatives:
                for _ in range(num_negatives):
                    neg = random.choice(negatives)
                    self.data.append((query, pos, neg))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# helper functions
def get_token_embeddings(text, tokenizer, model, device='cpu'):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)
    attention_mask = inputs['attention_mask'].squeeze(0).bool()
    return token_embeddings[attention_mask]

def get_doc_embedding(doc_id, metadata, tokenizer, model, device):
    text = f"{metadata[doc_id]['title']} {metadata[doc_id]['abstract']} Authors: {metadata[doc_id]['authors']}"
    return get_token_embeddings(text, tokenizer, model, device)

def colbert_score_from_emb(q_emb, d_emb):
    q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)
    d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)
    sim_matrix = torch.matmul(q_norm, d_norm.T)
    return sim_matrix.max(dim=1).values.sum()

# training function
def train_colbert_dual_encoder(df_query_train, metadata):
    dataset = ColBERTTripletDataset(df_query_train, metadata)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    query_model.train()
    doc_model.train()
    query_projection.train()

    optimizer = torch.optim.AdamW(
        list(query_model.parameters()) +
        list(doc_model.parameters()) +
        list(query_projection.parameters()),
        lr=LR
    )

    for epoch in range(EPOCHS):
        total_loss = 0.0
        for batch in tqdm(loader, desc=f"Epoch {epoch+1}"):
            queries, pos_ids, neg_ids = batch

            inputs = query_tokenizer(list(queries), return_tensors='pt', padding=True, truncation=True, max_length=512)
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            outputs = query_model(**inputs)
            q_emb_batch = outputs.last_hidden_state
            attention_mask = inputs["attention_mask"].bool()
            q_embs = [query_projection(emb[mask]) for emb, mask in zip(q_emb_batch, attention_mask)]

            score_pos_list = []
            score_neg_list = []

            for i in range(len(queries)):
                d_pos_emb = get_doc_embedding(pos_ids[i], metadata, doc_tokenizer, doc_model, DEVICE)
                d_neg_emb = get_doc_embedding(neg_ids[i], metadata, doc_tokenizer, doc_model, DEVICE)
                q_emb = q_embs[i]

                score_pos = colbert_score_from_emb(q_emb, d_pos_emb)
                score_neg = colbert_score_from_emb(q_emb, d_neg_emb)

                score_pos_list.append(score_pos)
                score_neg_list.append(score_neg)

            score_pos_batch = torch.stack(score_pos_list)
            score_neg_batch = torch.stack(score_neg_list)

            loss = F.relu(MARGIN + score_neg_batch - score_pos_batch).mean()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

    query_model.save_pretrained("finetuned_ctbert")
    query_tokenizer.save_pretrained("finetuned_ctbert")
    doc_model.save_pretrained("finetuned_scibert")
    doc_tokenizer.save_pretrained("finetuned_scibert")
    torch.save(query_projection.state_dict(), "query_projection.pt")


In [ ]:
train_colbert_dual_encoder(df_query_train, metadata)

In [ ]:
def rerank(df, metadata, tokenizer, q_model, d_model, save_name):
    device_q = next(q_model.parameters()).device
    device_d = next(d_model.parameters()).device
    df[save_name + '_scores'] = [[] for _ in range(len(df))]

    query_projection = nn.Linear(1024, 768)
    query_projection.load_state_dict(torch.load("query_projection.pt"))
    query_projection.to(device_q)
    query_projection.eval()

    doc_embeddings = {}
    for doc_id, data in metadata.items():
        emb = torch.load(data["path"], map_location="cpu")
        doc_embeddings[doc_id] = emb

    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            tweet_text = row['tweet_text']
            pre_ranked_docs = row['bm25_topk']

            q_emb = get_token_embeddings(tweet_text, tokenizer, q_model, device=device_q)
            q_emb = query_projection(q_emb)
            q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)

            scores = []
            for doc in pre_ranked_docs:
                emb = doc_embeddings[doc].to(device_d)
                length = metadata[doc]["length"]
                d_emb = emb[:length]
                d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)

                sim_matrix = torch.matmul(q_norm, d_norm.T)
                max_sim_per_q = sim_matrix.max(dim=1).values
                score = max_sim_per_q.sum().item()
                scores.append(score)

            df.at[idx, save_name + '_scores'] = scores

    def sort_docs_by_score(row):
        doc_ids = row['bm25_topk']
        scores = row[save_name + '_scores']
        sorted_docs = [doc for doc, _ in sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)]
        return sorted_docs

    df[save_name + '_topk'] = df.apply(sort_docs_by_score, axis=1)
    return df

In [ ]:
# pre-compute embeddings
metadata = pre_compute_embeddings("finetuned_scibert")

100%|██████████| 7718/7718 [03:40<00:00, 34.94it/s]


In [ ]:
# re-rank BM25 list for dev data
df_query_dev = rerank(df_query_dev, metadata, tokenizer, query_model, doc_model, "sciCtBERT-1")

100%|██████████| 1400/1400 [02:21<00:00,  9.89it/s]
